# Experimentos com o modelo pré-treinado do BERTimbau

## Testando o uso para limpeza do dataset de tweets das emoções de Ekman

In [ ]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline

In [ ]:
nome_modelo = "neuralmind/bert-large-portuguese-cased"
# nome_modelo = 'MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7'

model = AutoModelForPreTraining.from_pretrained(nome_modelo)
tokenizer = AutoTokenizer.from_pretrained(nome_modelo, do_lower_case=False)

classificador = pipeline("zero-shot-classification", model=nome_modelo, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-large-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to en

In [ ]:
import pandas as pd

path = '/content/drive/MyDrive/Colab Notebooks/tweets_ekman_2.csv'
df_ekman = pd.read_csv(path, sep=';')

df_feliz = df_ekman.loc[df_ekman['Sentimento'] == 'feliz']
df_feliz_amostra = df_feliz.sample(frac=0.3, random_state=0)

tweets_feliz = list(df_feliz_amostra['Texto'])

In [ ]:
sentimentos_previstos = []

for tweet in tweets_feliz:
  try:
    previsao = classificador(
        tweet,
        candidate_labels=['feliz', 'triste', 'raiva', 'nojo', 'medo', 'neutro']
    )

    valor_maximo = max(previsao['scores'])
    indice_valor_maximo = previsao['scores'].index(valor_maximo)

    sent_previsto = previsao['labels'][indice_valor_maximo]
  except Exception:
    sent_previsto = 'feliz'
  finally:
    sentimentos_previstos.append(sent_previsto)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
df_feliz_amostra['Previsao'] = sentimentos_previstos
df_feliz_amostra.to_csv('/content/drive/MyDrive/Colab Notebooks/tweets_feliz.csv')

In [ ]:
df_feliz_amostra.head()

,Texto,Sentimento,Previsao
7620,obrigada deus! tu és maravilhoso!!!!,feliz,feliz
9158,pedi logo o christian grey porque sou dessas ...,feliz,feliz
5549,nossa praia de perspectiva diferente! praia de...,feliz,triste
3421,"hoje, eu estou finalizando mais uma etapa do m...",feliz,triste
3639,"que minha coragem seja maior que o meu medo, e...",feliz,medo


In [ ]:
df_divergentes = df_feliz_amostra.loc[df_feliz_amostra['Sentimento'] != df_feliz_amostra['Previsao']]
df_divergentes.to_csv('/content/drive/MyDrive/Colab Notebooks/tweets_feliz_divergentes.csv', index=False)

In [ ]:
df_convergentes = df_feliz_amostra.loc[df_feliz_amostra['Sentimento'] == df_feliz_amostra['Previsao']]
df_convergentes.to_csv('/content/drive/MyDrive/Colab Notebooks/tweets_feliz_convergentes.csv', index=False)

In [ ]:
df_feliz_amostra.shape

(2840, 3)